# 6. LangChain 기초 실습

## 학습 목표
- LangChain으로 LLM을 추상화하여 사용하기
- Prompt Template, Output Parser, Chain 사용하기
- 메모리를 활용한 멀티턴 챗봇 만들기
- 스트리밍 응답 구현하기

## 1. 환경 설정

In [ ]:
!pip install langchain langchain-openai python-dotenv -q

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# find_dotenv(): 현재 디렉토리부터 상위로 올라가며 .env 파일을 자동으로 찾음
# 프로젝트 루트(/workspaces/study/.env)에 .env 파일을 두면 어디서든 동작
load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️ 프로젝트 루트에 .env 파일을 생성하고 OPENAI_API_KEY를 설정하세요")
else:
    print("✅ API 키 로드 완료")

## 2. LLM 추상화 클래스

LangChain은 다양한 LLM API를 통일된 인터페이스로 사용할 수 있게 해줍니다.

In [ ]:
from langchain_openai import ChatOpenAI

# LLM 인스턴스 생성
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=500
)

# 간단한 호출
response = llm.invoke("Python의 장점 3가지를 알려줘")
print(response.content)

### 다양한 LLM 제공자

같은 인터페이스로 다른 LLM도 사용 가능:

```python
# OpenAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4")

# Anthropic Claude
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(model="claude-3-sonnet-20240229")

# 사용법은 동일!
response = llm.invoke("Hello")
```

## 3. Prompt Template

프롬프트를 재사용 가능하게 만듭니다.

In [ ]:
from langchain.prompts import PromptTemplate

# 템플릿 정의
prompt = PromptTemplate.from_template(
    "{text}를 {language}로 번역해줘"
)

# 템플릿 포맷팅
formatted = prompt.format(text="Hello World", language="한국어")
print(formatted)
print()

# LLM에 전달
response = llm.invoke(formatted)
print(response.content)

### ChatPromptTemplate (시스템 + 사용자 메시지)

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친절한 {role} 선생님이야."),
    ("user", "{question}")
])

messages = prompt.format_messages(
    role="Python",
    question="데코레이터가 뭐야?"
)

response = llm.invoke(messages)
print(response.content)

## 4. Output Parser

LLM의 문자열 출력을 구조화된 데이터로 변환합니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# LLM 응답을 문자열로 파싱
response = llm.invoke("Hi there!")
parsed = parser.parse(response)
print(type(parsed))  # <class 'str'>
print(parsed)

### Pydantic Output Parser (구조화된 데이터)

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 출력 스키마 정의
class Person(BaseModel):
    name: str = Field(description="사람의 이름")
    age: int = Field(description="사람의 나이")
    occupation: str = Field(description="직업")

parser = PydanticOutputParser(pydantic_object=Person)

# 프롬프트에 포맷 지시 포함
prompt = PromptTemplate(
    template="다음 인물의 정보를 추출해줘:\n{format_instructions}\n\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | llm | parser
result = chain.invoke({"query": "철수는 30살이고 개발자로 일하고 있다."})

print(f"이름: {result.name}")
print(f"나이: {result.age}")
print(f"직업: {result.occupation}")

## 5. Chain (체인)

LCEL(LangChain Expression Language)로 컴포넌트를 연결합니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("{topic}에 대해 한 문장으로 설명해줘")
llm = ChatOpenAI()
parser = StrOutputParser()

# 체인 구성 (파이프 연산자)
chain = prompt | llm | parser

# 실행
result = chain.invoke({"topic": "LangChain"})
print(result)

### 순차 체인 (Sequential Chain)

In [ ]:
# 1단계: 아이디어 생성
idea_prompt = ChatPromptTemplate.from_template(
    "{industry} 분야의 스타트업 아이디어를 하나 제시해줘"
)
idea_chain = idea_prompt | llm | StrOutputParser()

# 2단계: 아이디어 분석
analysis_prompt = ChatPromptTemplate.from_template(
    "다음 아이디어를 분석해줘:\n{idea}\n\n장단점을 알려줘."
)
analysis_chain = analysis_prompt | llm | StrOutputParser()

# 전체 체인 조합
full_chain = (
    {"idea": idea_chain}
    | analysis_chain
)

result = full_chain.invoke({"industry": "헬스케어"})
print(result)

## 6. 메모리 (Memory)

LLM이 이전 대화를 기억하도록 합니다.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

# 대화 저장
memory.save_context(
    {"input": "내 이름은 철수야"},
    {"output": "안녕하세요 철수님!"}
)

memory.save_context(
    {"input": "내 이름이 뭐였지?"},
    {"output": "철수님이라고 하셨어요."}
)

# 메모리 확인
print(memory.load_memory_variables({}))

### ConversationChain (대화 체인)

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False  # True로 하면 내부 동작 확인 가능
)

# 대화
print("Bot:", conversation.predict(input="내 이름은 영희야"))
print()
print("Bot:", conversation.predict(input="내 이름이 뭐였지?"))
print()
print("Bot:", conversation.predict(input="방금 뭘 물어봤어?"))

## 7. 멀티턴 챗봇

완전한 대화형 챗봇을 만들어봅시다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# 설정
llm = ChatOpenAI(temperature=0.7)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

# 테스트 대화
test_messages = [
    "안녕! 나는 Python 배우는 중이야",
    "데코레이터가 뭔지 알려줄래?",
    "예제 코드도 보여줘",
    "내가 뭘 배우고 있다고 했지?"
]

for msg in test_messages:
    print(f"You: {msg}")
    response = conversation.predict(input=msg)
    print(f"Bot: {response}\n")

### Window Memory (최근 K개만 기억)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# 최근 2개 대화만 유지
window_memory = ConversationBufferWindowMemory(k=2)

conversation = ConversationChain(
    llm=llm,
    memory=window_memory,
    verbose=False
)

# 테스트
conversation.predict(input="메시지 1")
conversation.predict(input="메시지 2")
conversation.predict(input="메시지 3")  # 메시지 1은 잊혀짐
print(window_memory.load_memory_variables({}))

## 8. 스트리밍

응답을 실시간으로 받습니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("{topic}에 대한 짧은 이야기를 써줘")
parser = StrOutputParser()

chain = prompt | llm | parser

# 스트리밍
for chunk in chain.stream({"topic": "로봇"}):
    print(chunk, end="", flush=True)
print()

## 9. 실습 과제

### Level 1
1. PromptTemplate으로 다국어 번역기 만들기 (한국어→영어, 영어→일본어 등)
2. PydanticOutputParser로 영화 정보(제목, 감독, 장르, 개봉년도) 추출하기
3. ConversationChain으로 간단한 챗봇 만들기

In [ ]:
# 과제 1: 다국어 번역기
# TODO: 코드 작성


### Level 2
4. 순차 체인으로 "키워드 생성 → 블로그 제목 생성 → 본문 작성" 파이프라인
5. ConversationBufferWindowMemory로 최근 3개 대화만 기억하는 챗봇
6. 스트리밍으로 긴 답변을 실시간으로 출력하는 Q&A 시스템

In [ ]:
# 과제 2: 블로그 작성 파이프라인
# TODO: 코드 작성


### Level 3
7. Few-shot 프롬프트로 감성 분석기 만들기
8. 세션별로 대화를 관리하는 멀티 유저 챗봇
9. 병렬 체인으로 같은 질문을 여러 모델에 동시에 보내고 결과 비교

In [ ]:
# 과제 3: Few-shot 감성 분석
# TODO: 코드 작성


## 정리

LangChain의 장점:
- ✅ 통일된 인터페이스로 다양한 LLM 사용
- ✅ 프롬프트를 템플릿으로 재사용
- ✅ 출력을 구조화된 데이터로 파싱
- ✅ 체인으로 복잡한 워크플로우 구성
- ✅ 메모리로 대화 컨텍스트 유지

## 다음 단계

이제 Langfuse로 LangChain 애플리케이션을 모니터링해봅시다!

👉 [07_langfuse_practice.ipynb](07_langfuse_practice.ipynb)